In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, LSTM, Input, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, InceptionV3

# Set the path to your dataset
train_data_dir = r"C:\Users\asuto\Desktop\intern ship 2\lung detection\archive\archive\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Define the InceptionV3 model for feature extraction
cnn_input = Input(shape=(img_width, img_height, 3))
inception_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=cnn_input)

# Freeze the layers
for layer in inception_model.layers:
    layer.trainable = False

# Get the output of the InceptionV3 model
inception_output = inception_model.output

# Flatten the output
flatten_layer = Flatten()(inception_output)

# Define the DenseNet121 model for feature extraction
densenet_input = Input(shape=(img_width, img_height, 3))
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=densenet_input)

# Freeze the layers
for layer in densenet_model.layers:
    layer.trainable = False

# Get the output of the DenseNet121 model
densenet_output = densenet_model.output

# Concatenate the outputs of InceptionV3 and DenseNet121
concatenated_output = Concatenate()([flatten_layer, densenet_output])

# Define the LSTM model for sequence modeling
lstm_model = Sequential([
    LSTM(128, input_shape=(1, concatenated_output.shape[1])),
    Dense(len(class_names), activation='softmax')
])
lstm_output = lstm_model(tf.expand_dims(concatenated_output, axis=1))

# Combine the InceptionV3, DenseNet121, and LSTM models
combined_model = Model(inputs=[cnn_input, densenet_input], outputs=lstm_output)

# Compile and train the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=2)

# Save the trained model
combined_model.save('lung_detection_inceptionv3_densenet121.h5')


Found 9971 images belonging to 6 classes.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\asuto\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\asuto\AppData\Local\Temp\ipykernel_21284\2278482411.py", line 34, in <module>
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=cnn_input)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\asuto\AppData\Roaming\Python\Python311\site-packages\keras\src\applications\inception_v3.py", line 299, in InceptionV3
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\asuto\AppData\Roaming\Python\Python311\site-packages\keras\src\applications\inception_v3.py", line 433, in conv2d_bn
    x = layers.Conv2D(
        ^^^^^^^^^^^^^^
  File "C:\Users\asuto\AppData\Roaming\Python\Python311\site-packages\

In [ ]:
# Set the path to your test dataset
test_data_dir = r"C:\Users\asuto\Desktop\intern ship 2\lung detection\archive\archive\test-20230326T155708Z-001\test"

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Evaluate the model on the test data
loss, accuracy = combined_model.evaluate(
    test_generator,
    steps=len(test_generator))

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")


In [13]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\asuto\Desktop\intern ship 2\lung detection\archive\test-20230326T155708Z-001\test\COVID\COVID-19 (8).jpeg"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 0s 423ms/step
Predicted class: COVID
